# Bokeh Plot

In [21]:
import numpy as np

from bokeh.layouts import column
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource, Slider, CustomJS

from GreenFunctions_SC import GSC_matrix, FSC_matrix
from GreenFunctions_MTI import GMTI_normalBC, GMTI_modifiedBC, GMTI_analytical
from Perturbation_Theory import GMTI2_analytical, GMTI2_modifiedBC, FMTI2_analytical, FMTI2_modifiedBC


# coupling matrix
Gamma = np.matrix([[1,0],[0,1],[1,0],[0,1]])
# parameters
d = 100.; mu = 0.010; Delta = 0.005; omega = 0.0025; L = 0.005; z0 = 0.

# colors for plotting
colors = ["#0077FF", "#00CC00", "#FFD700", "#FF2200",]

# Enable Bokeh in Jupyter Notebook
output_notebook();

# real space lattice
x = np.linspace(0, d, 25)
# define Zeeman values
zeeman = np.arange(0., 0.010, 0.001)

Flist = [ [] for _ in range(16) ]
# loop over Zeeman
for L in zeeman:
    
    # compute pairing on lattice
    FMTI2 = [FMTI2_analytical(d=d, z=z, Z=z, z0=z0, L=L, mu=mu, Delta=Delta, omega=omega, Gamma=Gamma) for z in x]

    # loop over components
    for irow in range(4):
        for icol in range(4):
            # add F2 to F_list
            Flist[4*irow + icol].append( np.array([abs(f2[irow][icol]) for f2 in FMTI2]) )

# create 4 figures
p = figure(title="test", width=600, height=400, x_range=(0, d), y_range=(0,0.5))
 
source = ColumnDataSource(data={'x': x, 
                                'y1': Flist[0][0], 
                                'y2': Flist[1][0], 
                                'y3': Flist[2][0], 
                                'y4': Flist[3][0]})

# plot lines
p.line('x', 'y1', source=source, line_width=2, color='blue')
p.line('x', 'y2', source=source, line_width=2, color='green')
p.line('x', 'y3', source=source, line_width=2, color='gold')
p.line('x', 'y4', source=source, line_width=2, color='red')

# Create a Slider
slider = Slider(start=0, end=len(zeeman)-1, value=0, step=1, title="Zeeman")

# CustomJS callback for switching arrays
callback = CustomJS(args=dict(source=source, y_arrays=Flist, slider=slider), code="""
    var index = slider.value;
    var data = source.data;
    data['y1'] = y_arrays[0][index];
    data['y2'] = y_arrays[1][index];
    data['y3'] = y_arrays[2][index];
    data['y4'] = y_arrays[3][index];
    source.change.emit();
""")

slider.js_on_change('value', callback)

# Show the plot in Jupyter
show(column(p, slider))

Loading BokehJS ...

In [ ]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource, Slider, CustomJS
from bokeh.layouts import column
import numpy as np

# Enable Bokeh in Jupyter Notebook
output_notebook()

# Define x values
x = np.linspace(0, 10, 200)

# Define multiple y arrays (datasets)
y_arrays = [
    np.sin(x),          # Dataset 0
    np.sin(2*x),        # Dataset 1
    np.sin(3*x),        # Dataset 2
    np.sin(4*x),        # Dataset 3
]

# Convert to JavaScript-friendly format
y_arrays_js = [list(y) for y in y_arrays]

# Initial dataset
source = ColumnDataSource(data={'x': x, 'y': y_arrays[0]})

# Create figure
p = figure(title="Select Different Arrays with Slider", width=600, height=400)
p.line('x', 'y', source=source, line_width=2, color='blue')

# Create a Slider
slider = Slider(start=0, end=len(y_arrays)-1, value=0, step=1, title="Select Dataset")

# CustomJS callback for switching arrays
callback = CustomJS(args=dict(source=source, y_arrays=y_arrays_js, slider=slider), code="""
    var data = source.data;
    var index = slider.value;
    data['y'] = y_arrays[index];  // Update y data with selected dataset
    source.change.emit();
""")

slider.js_on_change('value', callback)

# Show the plot in Jupyter
show(column(p, slider))
